# Title
- When you feel you need to use function in general, then there is a natural barier here and you should write it into script and import it

<!--- ![alt text](/home/patrik/pcflow/drawio/images/code.drawio.png) -->





# Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from vis.deprecated_vis import *
import glob
from data.PATHS import DATA_PATH, TMP_VIS_PATH
from loss.flow import *

import argparse
from loss.flow import chamfer_distance_loss

ImportError: /home.dokt/vacekpa2/.local/lib/python3.10/site-packages/FastGeodisCpp.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE

# Structure Loss

# Use Case for NN Flow
Argoverse1 NSF - frame 2
<img src="/home/patrik/pcflow/use_cases/Smooth_NN_from_flow.png" width=600>

In [2]:
import matplotlib.pyplot as plt
from data.PATHS import TMP_VIS_PATH
import matplotlib.cm as cm

#
# color = cm.viridis(est_flow.norm(dim
#                                  =-1).detach().numpy()[0])
#
# plt.close()
# fig, axes = plt.subplots(1,2, figsize=(10,5))
# axes[0].plot(init_flow[0,:,0].detach(), init_flow[0,:,1].detach(), 'bo')
# axes[0].plot(pc2[0,:,0].detach(), pc2[0,:,1].detach(), 'ro')
#
# s = axes[1].scatter(est_flow[0,:,0].detach(), est_flow[0,:,1].detach(), c=per_point_f_loss.detach(), cmap=cm.viridis)
#
# cb = plt.colorbar(mappable=s, ax=axes[1])
# cb.set_label('Cbar Label Here')
#
# # axes[1].grid(True, which='both', axis='x', linewidth=1)
# plt.savefig(TMP_VIS_PATH + 'scatter.png')


In [12]:

# transfer args to pandas, to csv and back

import pandas as pd
args_list = [args, args]
values_list = []
names = [n[0] for n in args_list[0]._get_kwargs()]

for args in args_list:
    values = [n[1] for n in args._get_kwargs()]
    values_array = np.array((values))[None,:]
    values_list.append(values_array)

values_array = np.concatenate(values_list)

df = pd.DataFrame(values_array, columns=names, index=None)
df.to_csv('tst.csv')
# rewrite manually?

# Full loss example with animation
- write example points for use case [x]
- set up the losses for optimization of flow [x]
- store iterations in video []


In [54]:
# example points
pc1 = torch.tensor(((1.0,1,0), (1.3,1,0), (2.2,1,0))).unsqueeze(0)
pc2 = torch.tensor(((1.0,0.5,0), (1.8,0.6,0), (1.5, 0.4,0))).unsqueeze(0)
est_flow = torch.rand(pc1.shape) * 0.1
# est_flow = torch.ones_like(pc1, requires_grad=True)
# est_flow.retain_grad()
est_flow.requires_grad_()


tensor([[[0.0885, 0.0470, 0.0864],
         [0.0093, 0.0260, 0.0055],
         [0.0175, 0.0089, 0.0111]]], requires_grad=True)

In [86]:
plt.close()
plt.plot(pc1[0,:,0], pc1[0,:,1], 'bo')
plt.plot(pc2[0,:,0], pc2[0,:,1], 'ro')
plt.quiver(pc1[0,:,0], pc1[0,:,1], est_flow[0,:,0].detach(), est_flow[0,:,1].detach(), scale_units='xy',scale=1, color='g')

plt.axis('equal')

plt.savefig(TMP_VIS_PATH + 'points.png')


def store_toy_iter(pc1, pc2, est_flow, e):

    plt.close()
    plt.plot(pc1[0,:,0], pc1[0,:,1], 'bo')
    plt.plot(pc2[0,:,0], pc2[0,:,1], 'ro')
    plt.quiver(pc1[0,:,0], pc1[0,:,1], est_flow[0,:,0].detach(), est_flow[0,:,1].detach(), scale_units='xy', color='g', scale=1)

    plt.axis('equal')

    plt.savefig(TMP_VIS_PATH + f'video/{e:04d}_points.png')

In [198]:
# 2) smoothness propagation
# K_SM = 3
# _, NN_pc2, _ = knn_points(pc2, pc2, lengths1=None, lengths2=None, K=K_SM, norm=1)
#
# keep_ind = ind[ind[:,0] != pc2.shape[1] ,0]
#
# # print("\n\n Indices of pc2 for choosing only connections from NN(pc2) that has forward flow corespondences\n\n", keep_ind)
#
#
# # print("\n\n NN of pc2 that has correspondeces from flow\n\n", NN_pc2[0, keep_ind, :])
#
# # znamena, ze est flow body maji tyhle indexy pro body v pc2 a ty indexy maji mit stejne flow.
# n = NN_pc2[0, keep_ind, :]
#
# # beware of zeros
# connected_flow = vec[n] # N x KSmooth x 3 (fx, fy, fz)
# # print(connected_flow)
#
# # print(est_flow)
# prep_flow = est_flow[0].unsqueeze(1).repeat_interleave(repeats=K_SM, dim=1)
#
# # smooth it, should be fine
# flow_comparison = prep_flow - connected_flow
#
# # print(flow_comparison.shape)
# # print(flow_comparison)
# per_point_loss = flow_comparison.norm(dim=2).mean(dim=1)




In [55]:
# optimize  # add rigid kabsch to regularizer of NN?
# smoothness with option of pc2 smoothness
K_SM = 2
optimizer = torch.optim.SGD([est_flow], lr=0.1)
_, NN_pc2, _ = knn_points(pc2, pc2, lengths1=None, lengths2=None, K=K_SM, norm=1)



for e in range(100):

    _, forward_nn, _ = knn_points(pc1 + est_flow, pc2, lengths1=None, lengths2=None, K=1, norm=1)

    a = est_flow[0] # magnitude

    ind = forward_nn[0] # more than one?

    #if pc1 is bigger than pc2, then skip?
    if pc1.shape[1] < pc2.shape[1]:
        shape_diff = pc2.shape[1] - ind.shape[0] + 1 # one for dummy    # what if pc1 is bigger than pc2?
        a = F.pad(a, (0,0,0, shape_diff), mode='constant', value=0)
        a.retain_grad() # padding does not retain grad, need to do it manually. Check it

        ind = F.pad(ind, (0,0,0, shape_diff), mode='constant', value=pc2.shape[1])  # pad with dummy not in orig

    # storage of same points
    vec = torch.zeros(ind.shape[0], 3)

    vec = vec.scatter_reduce_(0, ind.repeat(1,3), a, reduce='mean', include_self=False)  # will do forward flow

    # rest is pc2 smoothness with pre-computed NN
    keep_ind = ind[ind[:,0] != pc2.shape[1] ,0]

    # znamena, ze est flow body maji tyhle indexy pro body v pc2 a ty indexy maji mit stejne flow.
    n = NN_pc2[0, keep_ind, :]

    # beware of zeros!!!
    connected_flow = vec[n] # N x KSmooth x 3 (fx, fy, fz)

    prep_flow = est_flow[0].unsqueeze(1).repeat_interleave(repeats=K_SM, dim=1) # correct

    # smooth it, should be fine
    flow_diff = prep_flow - connected_flow  # correct operation, but zeros makes problem

    occupied_mask = connected_flow.all(dim=2).repeat(3,1,1).permute(1,2,0)

    # occupied_mask
    per_flow_dim_diff = torch.masked_select(flow_diff, occupied_mask)

    # per_point_loss = per_flow_dim_diff.norm(dim=-1).mean()
    per_point_loss = (per_flow_dim_diff ** 2).mean()    # powered to 2 because norm will sum it directly

    chamf_loss, _, _ = chamfer_distance_loss(pc1 + est_flow, pc2, both_ways=False)

    loss = 10. * per_point_loss.mean() + chamf_loss
    # loss = chamf_loss

    loss.backward()

    store_toy_iter(pc1.detach(), pc2.detach(), est_flow.detach(), e)

    optimizer.step()
    optimizer.zero_grad()

    print(loss)



tensor(0.8017, grad_fn=<AddBackward0>)
tensor(0.7223, grad_fn=<AddBackward0>)
tensor(0.6569, grad_fn=<AddBackward0>)
tensor(0.6335, grad_fn=<AddBackward0>)
tensor(0.5929, grad_fn=<AddBackward0>)
tensor(0.5606, grad_fn=<AddBackward0>)
tensor(0.5279, grad_fn=<AddBackward0>)
tensor(0.4982, grad_fn=<AddBackward0>)
tensor(0.4540, grad_fn=<AddBackward0>)
tensor(0.4350, grad_fn=<AddBackward0>)
tensor(0.3837, grad_fn=<AddBackward0>)
tensor(0.3691, grad_fn=<AddBackward0>)
tensor(0.3210, grad_fn=<AddBackward0>)
tensor(0.3022, grad_fn=<AddBackward0>)
tensor(0.2843, grad_fn=<AddBackward0>)
tensor(0.2650, grad_fn=<AddBackward0>)
tensor(0.2479, grad_fn=<AddBackward0>)
tensor(0.2622, grad_fn=<AddBackward0>)
tensor(0.2455, grad_fn=<AddBackward0>)
tensor(0.2555, grad_fn=<AddBackward0>)
tensor(0.2409, grad_fn=<AddBackward0>)
tensor(0.2539, grad_fn=<AddBackward0>)
tensor(0.2415, grad_fn=<AddBackward0>)
tensor(0.2590, grad_fn=<AddBackward0>)
tensor(0.2428, grad_fn=<AddBackward0>)
tensor(0.2532, grad_fn=<A

In [84]:
# optimize  # add rigid kabsch to regularizer of NN?
# smoothness with option of pc2 smoothness

def forward_smoothness(pc1, pc2, est_flow, NN_pc2=None, K_SM=2, include_pc2_smoothness=True):

    if NN_pc2 is None and include_pc2_smoothness:
        # Compute NN_pc2
        _, NN_pc2, _ = knn_points(pc2, pc2, lengths1=None, lengths2=None, K=K_SM, norm=1)


    _, forward_nn, _ = knn_points(pc1 + est_flow, pc2, lengths1=None, lengths2=None, K=1, norm=1)

    a = est_flow[0] # magnitude

    ind = forward_nn[0] # more than one?

    if pc1.shape[1] < pc2.shape[1]:
        shape_diff = pc2.shape[1] - ind.shape[0] + 1 # one for dummy    # what if pc1 is bigger than pc2?
        a = F.pad(a, (0,0,0, shape_diff), mode='constant', value=0)
        a.retain_grad() # padding does not retain grad, need to do it manually. Check it

        ind = F.pad(ind, (0,0,0, shape_diff), mode='constant', value=pc2.shape[1])  # pad with dummy not in orig

    # storage of same points
    vec = torch.zeros(ind.shape[0], 3)

    # this is forward flow withnout NN_pc2 smoothness
    vec = vec.scatter_reduce_(0, ind.repeat(1,3), a, reduce='mean', include_self=False)

    forward_flow_loss = torch.nn.functional.mse_loss(vec[ind[:,0]], a, reduction='none').mean(dim=-1)

    if include_pc2_smoothness:
        # rest is pc2 smoothness with pre-computed NN
        keep_ind = ind[ind[:,0] != pc2.shape[1] ,0]

        # znamena, ze est flow body maji tyhle indexy pro body v pc2 a ty indexy maji mit stejne flow.
        n = NN_pc2[0, keep_ind, :]

        # beware of zeros!!!
        connected_flow = vec[n] # N x KSmooth x 3 (fx, fy, fz)

        prep_flow = est_flow[0].unsqueeze(1).repeat_interleave(repeats=K_SM, dim=1) # correct

        # smooth it, should be fine
        flow_diff = prep_flow - connected_flow  # correct operation, but zeros makes problem

        occupied_mask = connected_flow.all(dim=2).repeat(3,1,1).permute(1,2,0)

        # occupied_mask
        per_flow_dim_diff = torch.masked_select(flow_diff, occupied_mask)

        # per_point_loss = per_flow_dim_diff.norm(dim=-1).mean()
        NN_pc2_loss = (per_flow_dim_diff ** 2).mean()    # powered to 2 because norm will sum it directly

    else:
        NN_pc2_loss = torch.tensor(0.)

    return forward_flow_loss.mean(), forward_flow_loss, NN_pc2_loss


# asi good

# todo visualize connections of NN

In [88]:
# forward_smoothness(pc1, pc2, est_flow, NN_pc2=None, K_SM=2, include_pc2_smoothness=True)



tensor([[0],
        [2],
        [1]])


(tensor(0., grad_fn=<MeanBackward0>),
 tensor([0., 0., 0.], grad_fn=<MeanBackward1>),
 tensor(0.0048, grad_fn=<MeanBackward0>))